In [22]:
# Importações necessárias
import os
import pandas as pd
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_vertexai import VertexAIEmbeddings
from langchain_community.vectorstores import FAISS

In [23]:
# Definir o diretório dos documentos
data_dir = "rag_project/data/raw"

In [24]:
# Carregar os documentos
def load_documents(directory):
    """
    Carrega todos os documentos de texto de um diretório.
    
    Args:
        directory (str): Caminho para o diretório contendo os documentos.
        
    Returns:
        list: Lista de documentos carregados.
    """
    print(f"Carregando documentos do diretório: {directory}")
    
    # Usar o DirectoryLoader do LangChain para carregar todos os arquivos .txt
    loader = DirectoryLoader(
        directory,
        glob="**/*.txt",
        loader_cls=TextLoader,
        loader_kwargs={"encoding": "utf-8"}
    )
    
    documents = loader.load()
    print(f"Carregados {len(documents)} documentos.")
    
    return documents

# Segmentar os documentos em chunks menores
def split_documents(documents, chunk_size=1000, chunk_overlap=200):
    """
    Divide os documentos em chunks menores para processamento.
    
    Args:
        documents (list): Lista de documentos a serem divididos.
        chunk_size (int): Tamanho de cada chunk em caracteres.
        chunk_overlap (int): Sobreposição entre chunks consecutivos.
        
    Returns:
        list: Lista de chunks de documentos.
    """
    print(f"Segmentando documentos em chunks de {chunk_size} caracteres com sobreposição de {chunk_overlap}.")
    
    # Usar o RecursiveCharacterTextSplitter do LangChain para dividir os documentos
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n", "\n", " ", ""]
    )
    
    chunks = text_splitter.split_documents(documents)
    print(f"Documentos segmentados em {len(chunks)} chunks.")
    
    return chunks

In [25]:
# Executar o carregamento e segmentação
documents = load_documents(data_dir)
chunks = split_documents(documents)

Carregando documentos do diretório: rag_project/data/raw
Carregados 9 documentos.
Segmentando documentos em chunks de 1000 caracteres com sobreposição de 200.
Documentos segmentados em 23 chunks.


In [26]:
# Inicializar o modelo de embedding da Vertex AI
def initialize_embeddings():
    """
    Inicializa o modelo de embedding da Vertex AI.
    
    Returns:
        VertexAIEmbeddings: Instância do modelo de embedding.
    """
    print("Inicializando o modelo de embedding da Vertex AI...")
    
    embeddings = VertexAIEmbeddings(
      model_name="text-embedding-004",
      project="ccdsiaa",
      location="us-central1"
    )
    
    return embeddings

In [27]:
# Criar o índice FAISS
def create_vector_index(chunks, embeddings):
    """
    Cria um índice vetorial FAISS a partir dos chunks de documentos.
    
    Args:
        chunks (list): Lista de chunks de documentos.
        embeddings: Modelo de embedding a ser utilizado.
        
    Returns:
        FAISS: Índice vetorial FAISS.
    """
    print("Criando índice vetorial FAISS...")
    
    # Criar o índice FAISS
    vector_index = FAISS.from_documents(chunks, embeddings)
    
    print("Índice vetorial criado com sucesso.")
    
    return vector_index

In [28]:
# Salvar o índice FAISS
def save_vector_index(vector_index, path):
    """
    Salva o índice vetorial FAISS em disco.
    
    Args:
        vector_index: Índice vetorial FAISS.
        path (str): Caminho para salvar o índice.
    """
    print(f"Salvando índice vetorial em {path}...")
    
    # Criar o diretório se não existir
    os.makedirs(os.path.dirname(path), exist_ok=True)
    
    # Salvar o índice
    vector_index.save_local(path)
    
    print("Índice vetorial salvo com sucesso.")

In [1]:

!gcloud auth application-default login

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=kWmZkFGHvjq2WzK0RHQgiZmQCLtv71&access_type=offline&code_challenge=mGkPS7cvlGBglBqxEipaJiV4ZxPuxJNLZpdB7jj4ce0&code_challenge_method=S256


Credentials saved to file: [C:\Users\marri\AppData\Roaming\gcloud\application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).

Quota project "ccdsiaa" was added to ADC which can be used by Google client libraries for billing and quota. Note that some services may still bill the project owning the resource.


In [30]:
# Executar a geração de embeddings e indexação
embeddings = initialize_embeddings()
vector_index = create_vector_index(chunks, embeddings)
save_vector_index(vector_index, "rag_project/models/faiss_index")

Inicializando o modelo de embedding da Vertex AI...
Criando índice vetorial FAISS...
Índice vetorial criado com sucesso.
Salvando índice vetorial em rag_project/models/faiss_index...
Índice vetorial salvo com sucesso.


In [31]:
# Carregar o índice FAISS (se necessário)
def load_vector_index(path, embeddings):
    """
    Carrega um índice vetorial FAISS do disco.
    
    Args:
        path (str): Caminho do índice.
        embeddings: Modelo de embedding a ser utilizado.
        
    Returns:
        FAISS: Índice vetorial FAISS.
    """
    print(f"Carregando índice vetorial de {path}...")
    
    # Carregar o índice
    vector_index = FAISS.load_local(path, embeddings)
    
    print("Índice vetorial carregado com sucesso.")
    
    return vector_index

In [ ]:
# Inicializar o LLM da Vertex AI
def initialize_llm():
    """
    Inicializa o modelo de linguagem da Vertex AI.
    
    Returns:
        VertexAI: Instância do modelo de linguagem.
    """
    print("Inicializando o modelo de linguagem da Vertex AI...")
    
    llm = VertexAI(
        model_name="gemini-2.5-flash-preview-05-20",
        max_output_tokens=1024,
        temperature=0.2,
        top_p=0.8,
        top_k=40,
        project="ccdsiaa",
        location="us-central1"
    )
    
    return llm

In [33]:
# Criar o template de prompt
def create_prompt_template():
    """
    Cria um template de prompt para o LLM com uma persona proativa.
    
    Returns:
        PromptTemplate: Template de prompt.
    """
    template = """
    Aja como o "Guia UTF", o assistente virtual oficial da UTFPR-CM (Universidade Tecnológica Federal do Paraná campus Campo Mourão). Você é um especialista nos regulamentos, datas e contatos da universidade. Seu objetivo é fornecer respostas claras, precisas e úteis para os estudantes.

    **Suas regras de ouro são:**
    1.  **Baseie-se nos Fatos:** Responda UNICAMENTE com base no CONTEXTO fornecido. Não invente informações, datas ou processos.
    2.  **Seja Honesto:** Se a resposta não estiver no CONTEXTO, diga explicitamente: "Não encontrei essa informação nos documentos disponíveis."
    3.  **Seja Útil Mesmo Sem a Resposta:** Após admitir que não sabe, sempre tente ajudar. Sugira um próximo passo, como: "Para informações sobre isso, o ideal é contatar diretamente o DERAC (Departamento de Registros Acadêmicos) no Bloco X" ou "Recomendo que você converse com o coordenador do seu curso para um conselho mais personalizado."
    4.  **Tom e Estilo:** Mantenha um tom amigável, encorajador e respeitoso. Organize a informação em listas ou parágrafos curtos para facilitar a leitura.

    CONTEXTO:
    {context}
    
    PERGUNTA DO ALUNO:
    {question}
    
    SUA RESPOSTA (COMO GUIA UTF):
    """
    
    prompt_template = PromptTemplate(
        template=template,
        input_variables=["context", "question"]
    )
    
    return prompt_template

In [34]:
# Criar a função de recuperação e geração
def create_rag_chain(vector_index, llm, prompt_template):
    """
    Cria uma cadeia de recuperação e geração (RAG).
    
    Args:
        vector_index: Índice vetorial FAISS.
        llm: Modelo de linguagem.
        prompt_template: Template de prompt.
        
    Returns:
        RetrievalQA: Cadeia de recuperação e geração.
    """
    print("Criando cadeia RAG...")
    
    # Criar o retriever
    retriever = vector_index.as_retriever(
        search_type="similarity",
        search_kwargs={"k": 5}  # Recuperar os 5 chunks mais relevantes
    )
    
    # Criar a cadeia RAG
    rag_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",  # "stuff" combina todos os documentos em um único prompt
        retriever=retriever,
        return_source_documents=True,
        chain_type_kwargs={"prompt": prompt_template}
    )
    
    print("Cadeia RAG criada com sucesso.")
    
    return rag_chain

In [35]:
# Função principal para responder perguntas
def answer_question(question, rag_chain):
    """
    Responde a uma pergunta utilizando a cadeia RAG.
    
    Args:
        question (str): Pergunta do usuário.
        rag_chain: Cadeia RAG.
        
    Returns:
        dict: Resposta e documentos fonte.
    """
    print(f"Processando pergunta: {question}")
    
    # Executar a cadeia RAG
    result = rag_chain({"query": question})
    
    return {
        "pergunta": question,
        "resposta": result["result"],
        "documentos_fonte": result["source_documents"]
    }